# 데이터 전처리

데이터 전처리는 머신러닝 알고리즘만큼 중요하다.  
머신러닝 알고리즘은 데이터에 기반하고 있기 때문에 Garbage In, Garbage Out  

결측치는 허용되지 않는다.  
즉, 결측치를 다른값으로 채워 넣어야 한다.  
결측치 처리방법은 경우에 따라 다르다.  
변수의 결측치가 적다면 변수의 평균값으로 대체 가능하지만,  
결측치가 대부분이라면, 해당 변수는 드롭하는 것이 좋다.  
가장 어려운 점은 특정되지는 않으나 일정 수준 이상의 결측치를 가진다면,  
그리고 중요도가 높은 변수라서 단순히 평균치로 대체할 경우  
예측 왜곡이 심할 수 있다면 다른 대체 방법을 찾아야 한다.  

사이킷런의 머신러닝 알고리즘은 문자열 값을 입력값으로 허용하지 않기 때문에  
모든 문자영 값을 인코딩하여 수치값으로 변환 해야 한다.  
불필요한 피처라고 판단되면 삭제하는 것이 좋다.  
예를 들어 주민번호나 단순 문자열 아이디의 경우 단순히 데이터 로우를 식별하는 용도로,  
예측에 중요한 요소가 아니고, 알고리즘을 오히려 복잡하게 만들어 예측 성능을 떨어뜨린다.

## 데이터 인코딩

머신러닝을 위한 대표적인 인코딩 방식은 라벨 인코딩과 원-핫 인코딩이 있다.  
라벨 인코딩은 다항변수를 코드형 숫자값으로 변환하는 것이다.  
주의점으로 '01', '02' 같은 코드값은 문자열이므로, 1, 2 같은 숫자형으로 변환 해야 함.  

### 라벨 인코딩

사이킷런의 라벨 인코딩은 라벨인코더 클래스로 구현한다.  
라벨인코더를 객체로 생성한 후, fit()와 transform()을 호출해 라벨인코딩을 수행한다.

In [1]:
from sklearn.preprocessing import LabelEncoder

items=['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

#LabelEncoder를 객체로 생성한 후, fit()과 transform()으로 레이블 인코딩 수행.
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값:', labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


각각 0, 1, 2, 3, 4, 5로 변환됐다.  
위 예제는 데이터가 적어 직관적으로 알 수 있지만,  
많은 경우는 알 수 없기에 라벨인코더 객체에 classes_ 로 확인한다.

In [2]:
print('인코딩 클래스:', encoder.classes_)

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


classes_는 0번부터 변환된 인코딩 값의 원본값을 가짐  
즉, TV는 0, 냉장고 1, 믹서 2, 선풍기 3, 전자렌지 4, 컴퓨터 5  
inverse_transform()로 다시 디코딩이 가능하다.

In [3]:
print('디코딩 변환값:', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

디코딩 변환값: ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


몇몇 머신러닝 알고리즘은 레이블 인코딩이 적용할 경우 예측 성능이 떨어지는 경우가 발생한다.  
이는 숫자 값의 경우 크고 작음에 대한 특성이 작용하기 때문이다.  
예를 들어, 냉장고가 1, 믹서가 2로 변환되면 1보다 2가 더 큰 값 이므로  
특정 머신러닝 알고리즘에서 가중치가 더 부여되거나 더 중요하게 인식될 가능성이 있다.
하지만 냉장고와 믹서의 변환 값은 단순 코드기 때문에 숫자 값에 따른 순서나 중요도로 인식돼서는 안 된다.  
이러한 특성 때문에 라벨 인코딩은 선형회귀와 같은 머신러닝 알고리즘에는 적용하지 말아야한다.  
이러한 문제점을 해결하기 위해 원-핫 인코딩을 사용한다.
트리 계열의 알고리즘은 이러한 특성을 반영하지 않으므로 라벨 인코딩을 사용하여도 문제가 없다.  

### 원-핫 인코딩

원-핫 인코딩은 행 형태로 돼 있는 피처의 고유 값을 열 형태로 차원을 변환한 뒤  
고유 값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시한다.

원-핫 인코딩은 사이킷런에서 OneHotEncoder 클래스로 변환이 가능하다.  
단, LabelEncoder와 다르게 주의점이 있다. 입력값으로 2차원 데이터가 필요하고,  
OneHotEncoder를 이용해 변환한 값이 희소 행렬 형태이므로 이를 다시  
toarray() 메서드를 이용해 밀집행렬로 변환해야 한다는 것이다.  


In [4]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 2차원 ndarray로 변환
items = np.array(items).reshape(-1, 1)

# 원-핫 인코딩 적용
oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

# OneHotEncoder로 변환한 결과는 희소행렬이므로 toarray()를 이용해 밀집행렬로 변환
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(8, 6)


8개의 레코드와 1개의 칼럼을 가진 원본 데이터가 8개의 레코드와 6개의 칼럼을 가진 데이터로 변환되었다.  
  
판다스에는 원-핫 인코딩을 더 쉽게 지원하는 get_dummies() API가 있다.  
사이킷런의 OneHotEncoder와 다르게 문자열 카테고리 값을 숫자 형으로 변환할 필요 없이 바로 변환할 수 있다.

In [21]:
import pandas as pd

df = pd.DataFrame({'item':['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']})
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


In [23]:
import pandas as pd

df = pd.DataFrame(items)
pd.get_dummies(df)

,0_TV,0_냉장고,0_믹서,0_선풍기,0_전자레인지,0_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 피쳐 스케일링과 정규화

피쳐 스케일링은 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업이다.  
대표적인 방법으로 표준화와 정규화가 있다.  

표준화는 데이터의 피쳐 각각이 평균이 0이고 분산이 1인  
가우시안 정규분포를 가진 값으로 변환하는 것을 의미한다.  
  
정규화는 서로 다른 피처의 크기를 통일하기 위해 크기를 변환해주는 것이다.

### StandardScaler

StandardScaler는 개별 피처를 평균이 0이고, 1인 값으로 변환해 준다.  
이렇게 가우시안 정규 분포를 가질 수 있도록 데이터를 변환하는 것은 몇몇 알고리즘에서 매우 중요하다.  
특히 사이킷런에서 구현한 RBF 커널을 이용한 SVM, 선형회귀, 로지스틱 회귀는  
사전에 표준화를 적용하는 것은 예측 성능 향상에 중요한 요소가 될 수 있다.

In [6]:
from sklearn.datasets import load_iris

#붓꽃 데이터 세트를 로딩하고 DataFrame으로 변환
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

print('feature 들의 평균 값')
print(iris_df.mean())
print('\nfeature 들의 분산 값')
print(iris_df.var())

feature 들의 평균 값
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature 들의 분산 값
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


이제 StandardScaler를 이용해 각 피처를 한 번에 표준화해 변환한다.  
StandardScaler 객체를 생성한 후에 fit()과 transform() 메서드에  
변환 대상 피처 데이터 세트를 입력하고 호출하면 간단하게 변환된다.  
transform()을 호출할 때 스케일 변환된 데이터 세트가 넘파이의 ndarray이므로  
이를 DataFrame으로 변환해 평균값과 분산 값을 다시 확인한다.

In [7]:
from sklearn.preprocessing import StandardScaler

# StandardScaler객체 생성
scaler = StandardScaler()
# StandardScaler로 데이터 세트 변환. fit()와 transform()호출.
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform()시 스케일 변환된 데이터 세트가 NumPy ndarray로 반환돼 이를 DataFrame으로 반환돼 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature들의 평균 값')
print(iris_df_scaled.mean())
print('\nfeature들의 분산 값')
print(iris_df_scaled.var())

feature들의 평균 값
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

feature들의 분산 값
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


모든 칼럼의 평균이 0에 아주 가까운 값으로, 그리고 분산은 1에 가까운 값으로 변환됨을 알 수 있다.

### MinMaxScaler

MinMaxScaler는 데이터값을 0과 1 사이의 범위값으로 변환한다.  
데이터의 분포가 가우시안 분포가 아닐 경우 MinMaxScaler를 적용해 볼 수 있다.

In [8]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler객체 생성
scaler = MinMaxScaler()
# MinMaxScaler로 데이터 세트 변환. fit()와 transform()호출.
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform()시 스케일 변환된 데이터 세트가 NumPy ndarray로 반환돼 이를 DataFrame으로 반환돼 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature들의 최소값')
print(iris_df_scaled.min())
print('\nfeature들의 최대값')
print(iris_df_scaled.max())

feature들의 최소값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature들의 최대값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


모든 변수에 0에서 1사이의 값으로 변환되었음을 알 수 있다.

### 학습데이터와 테스트 데이터의 스케일링 변환시 유의점

StandardScaler나 MinMaxScaler와 같은 Scaler 객체를 이용해 데이터의 스케일링 변환 시  
fit(), transform(), fit_transform() 메서드를 이용한다.  
fit()은 데이터 변환을 위한 기준 정보 설정을 적용하며  
transform()은 설정된 정보를 이용해 데이터를 변환합니다.  
fit_transform()은 fit()과 transform()을 한 번에 적용하는 기능을 수행한다.  

하지만 학습 데이터 세트와 테스트 데이터 세트에 fit()과 transform()을 적용할 때 주의가 필요하다.  
학습 데이터로 fit()이 적용된 스케일링 기준 정보를 그대로 테스트 데이터에 적용해야 하며,  
그렇지 않고 테스트 데이터로 다시 새로운 스케일링 기준 정보를 만들게 되면  
학습 데이터와 테스트 데이터의 스케일링 기준 정보가 서로 달라지기 때문에 올바른 예측결과를 도출하지 못할 수 있다.

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습데이터는 0에서 10까지, 테스트 데이터는 0부터 5까지 값을 가지는 데이터 세트로 생성
# Scaler 클래스의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape(-1,1)로 차원 변경
train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

In [10]:
# MinMaxScaler 객체에 별도의 feature_range 파라미터 값을 지정하지 않으면 0~1 값으로 변환
scaler = MinMaxScaler()

# fit()하게 되면 train_array 데이터의 최솟값이 0, 최대값이 10으로 설정
scaler.fit(train_array)

# 1/10 scale로 train_array 데이터 변환함. 원본 10 -> 1로 변환됨
train_scaled = scaler.transform(train_array)

print('원본 train_array 데이터', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터', np.round(train_scaled.reshape(-1), 2))

원본 train_array 데이터 [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [12]:
# MinMaxScaler에 test_array를 fit()하게 되면 원본 데이터의 최솟값이 0, 최대값이 5으로 설정
scaler.fit(test_array)

# 1/5 scale로 test_array 데이터 변환함. 원본 5 -> 1로 변환됨
test_scaled = scaler.transform(test_array)

print('원본 test_array 데이터', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터', np.round(test_scaled.reshape(-1), 2))

원본 test_array 데이터 [0 1 2 3 4 5]
Scale된 test_array 데이터 [0.  0.2 0.4 0.6 0.8 1. ]


출력 결과를 확인하면 학습 데이터와 테스트 데이터의 스케일링이 맞지 않음을 알 수 있다.  
테스트 데이터의 경우는 원본값 1은 0.2로, 원본값 5는 1로 변환됐다.  
앞서 학습 데이터는 스케일링 변환으로 원본값 2가 0.2로 변환됐고, 원본값 10 이 1로 변환됐다.  
이러면 학습 데이터와 테스트 데이터의 서로 다른 원본값이 동일한 값으로 변환되는 결과를 초래한다.  
즉, 테스트 데이터에 다시 fit()을 적용해서는 안 되며, 학습 데이터로 이미 fit()이 적용된  
Scaler 객체를 이용해 transform()으로 변환돼야 한다. 

In [13]:
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('원본 train_array 데이터', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터', np.round(train_scaled.reshape(-1), 2))

# test_array에 Scale 변환을 할 때는 반드시 fit()을 호출하지 않고 transform()만으로 변환해야 함
test_scaled = scaler.transform(test_array)
print('원본 test_array 데이터', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터', np.round(test_scaled.reshape(-1), 2))

원본 train_array 데이터 [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
원본 test_array 데이터 [0 1 2 3 4 5]
Scale된 test_array 데이터 [0.  0.1 0.2 0.3 0.4 0.5]


fit_transform()도 마찬가지다.  
fit_transform()은 fit()과 transform()을 순차적으로 실행하므로 학습 데이터는 상관없지만 테스트 데이터에는 절대 사용하면 안 된다.  
이럴 땐 학습과 테스트 데이터 세트로 분리하기 전, 먼저 전체 데이터 세트에 스케일링을 한 뒤 분리하는 것이 더 바람직하다.  
  
학습 데이터와 테스트 데이터의 fit(), transform() . fit_transform()을 이용해 스케일링 변환 시 유의할 점을 요약하면 다음과 같다.  
1. 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 학습과 테스트 데이터로 분리
2. 1이 안 되면 테스트 데이터에는 fit(), fit_transform()을 쓰지말고, 학습 데이터로 fit()된 Scaler 객체를 가져와 transform( )으로 변환 